In [3]:
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext

In [ ]:
# using the boto3 the get data from S3
import boto3
import io
import pandas as pd
session = boto3.Session()
credentials = session.get_credentials()
credentials = credentials.get_frozen_credentials()
access_key = '*******'
secret_key = '**********'
s3 = session.resource('s3')
client = boto3.client('s3', aws_access_key_id=access_key,
      aws_secret_access_key=secret_key)

In [3]:
# to read the csv, we can use this code
bucket = s3.Bucket('Bucket')
s3_file_key = 'file/name.csv'
#bucket = 'Bucket'
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket.name, Key=s3_file_key)
millimanintelli_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
millimanintelli_df.head()  

,PersonID,FirstName,LastName,DOB,SSN,Gender,ZipCode
0,42890708,Belinda,Harper,1980-05-16,563613038,F,93505.0
1,42890709,Alberto,Torres,1964-12-12,611626678,M,90638.0
2,42890710,Kiran,Mann,1970-08-29,,F,95337.0
3,42890712,Tazari,Woodward,1998-11-05,607114739,F,91764.0
4,42890713,Donna,Garvey,1947-03-06,523669738,F,80014.0


In [4]:
# read list the files in S3 subdirectory using Python
import boto3
s3 = boto3.resource('s3')

## Bucket to use
bucket = s3.Bucket('Bucket')  # which Bucket

## List objects within a given prefix
for obj in bucket.objects.filter(Delimiter='/', Prefix='file/'): # Prefix is which directory we need to find
  print (obj.key)

millimanintelli/
millimanintelli/milliam_matched_48k.csv_$folder$
millimanintelli/millimanintelli.csv


In [5]:
from pyspark.sql import * # we can also use the spark to read the csv on our computer
#df = sql_c.read.csv('/Users/katie.xia/Desktop/project-classifier/nwdata.csv', inferSchema = True, header = True)

In [6]:
# use the spark to connect s3 and read s3's file (the set up)
import os
import pyspark
from pyspark.sql import SparkSession
import configparser

access_id = "********"
access_key ="********"
# set spark
spark = SparkSession.builder.master('local').appName("test").getOrCreate()
sc=spark.sparkContext
hadoop_conf=sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoop_conf.set("fs.s3n.awsAccessKeyId", access_id)
hadoop_conf.set("fs.s3n.awsSecretAccessKey", access_key)

In [7]:
#df_song = spark.read.csv('s3a://sikkasparkdata/millimanintelli/millimanintelli.csv')

In [8]:
# use spark to read csv in s3
millimanintelli=spark.read.csv('s3n://Bucket/file/file1.csv',inferSchema = True, header = True)

In [9]:
millimanintelli.head(5)

[Row(PersonID=42890708, FirstName='Belinda', LastName='Harper', DOB=datetime.datetime(1980, 5, 16, 0, 0), SSN='563613038', Gender='F', ZipCode='93505'),
 Row(PersonID=42890709, FirstName='Alberto', LastName='Torres', DOB=datetime.datetime(1964, 12, 12, 0, 0), SSN='611626678', Gender='M', ZipCode='90638'),
 Row(PersonID=42890710, FirstName='Kiran', LastName='Mann', DOB=datetime.datetime(1970, 8, 29, 0, 0), SSN='         ', Gender='F', ZipCode='95337'),
 Row(PersonID=42890712, FirstName='Tazari', LastName='Woodward', DOB=datetime.datetime(1998, 11, 5, 0, 0), SSN='607114739', Gender='F', ZipCode='91764'),
 Row(PersonID=42890713, FirstName='Donna', LastName='Garvey', DOB=datetime.datetime(1947, 3, 6, 0, 0), SSN='523669738', Gender='F', ZipCode='80014')]

In [10]:
# create the view of sql
millimanintelli.createOrReplaceTempView("table1")
#df2 = spark.sql("SELECT * from millimanintell limit 5")

In [11]:
# use sql in spark
df1 = spark.sql("SELECT * from table1 limit 5")
df1.show()  # millimanintelli in sql (table 1)

+--------+---------+--------+-------------------+---------+------+-------+
|PersonID|FirstName|LastName|                DOB|      SSN|Gender|ZipCode|
+--------+---------+--------+-------------------+---------+------+-------+
|42890708|  Belinda|  Harper|1980-05-16 00:00:00|563613038|     F|  93505|
|42890709|  Alberto|  Torres|1964-12-12 00:00:00|611626678|     M|  90638|
|42890710|    Kiran|    Mann|1970-08-29 00:00:00|         |     F|  95337|
|42890712|   Tazari|Woodward|1998-11-05 00:00:00|607114739|     F|  91764|
|42890713|    Donna|  Garvey|1947-03-06 00:00:00|523669738|     F|  80014|
+--------+---------+--------+-------------------+---------+------+-------+



In [12]:
# use spark to read text file in spark
from pyspark.sql.types import *
schema_notetext = StructType([
   StructField("PatID", StringType()),
   StructField("Fname", StringType()),
   StructField("Lname", StringType()),
   StructField("Bdate", DateType()),
   StructField("Add1", StringType()),
   StructField("Add2", StringType()),
   StructField("City", StringType()),
   StructField("State", StringType()),
   StructField("Zip", StringType()),
   StructField("SSN_second", StringType()),
   StructField("NoteType", StringType()),
   StructField("NoteName", StringType()),
   StructField("NoteValue", StringType()),
   StructField("DateCreated", DateType()),
   StructField("DateUpdated", DateType()),
   StructField("Custid", IntegerType()),
   StructField("PracticeID", StringType()),
   StructField("MasterID", StringType())
   ])
df_notetext = spark.read \
  .format("com.databricks.spark.csv") \
  .option("header", "true") \
  .option("delimiter",",") \
  .option('ignoreLeadingWhiteSpace',"true") \
  .option('ignoreTrailingWhiteSpace', "true") \
  .option("treatEmptyValuesAsNulls","true") \
  .option("treatParseExceptionAsNull","true") \
  .option("insertNullOnErrors","true") \
  .option("mode", "DROPMALFORMED") \
  .schema(schema_notetext) \
  .load("s3n://sikkasparkdata/dental/patientmedicalnotes/")

In [13]:
# create table in spark sql and use sql in spark
df_notetext.createOrReplaceTempView("table2")
df2 = spark.sql("SELECT * from table2 limit 10") # df2 is table2 is df_notetext in sql

In [ ]:
df2.show()